In [1]:
#!/usr/bin/python3
import sys
import numpy as np

import cv2
from post_24 import compute
import time
import csv

###############################################################################
# Utility code from
# http://git.io/vGi60A
# Thanks to author of the sudoku example for the wonderful blog posts!
###############################################################################

def rectify(h):
    h = h.reshape((4, 2))
    hnew = np.zeros((4, 2), dtype=np.float32)

    add = h.sum(1)
    hnew[0] = h[np.argmin(add)]
    hnew[2] = h[np.argmax(add)]

    diff = np.diff(h, axis=1)
    hnew[1] = h[np.argmin(diff)]
    hnew[3] = h[np.argmax(diff)]
    return hnew


###############################################################################
# Image Matching
###############################################################################

def preprocess(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 2)
    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 1)
    return thresh

def imgdiff(img1, img2):
    img1 = cv2.GaussianBlur(img1, (5, 5), 5)
    img2 = cv2.GaussianBlur(img2, (5, 5), 5)
    diff = cv2.absdiff(img1, img2)
    diff = cv2.GaussianBlur(diff, (5, 5), 5)
    flag, diff = cv2.threshold(diff, 200, 255, cv2.THRESH_BINARY)
    return np.sum(diff)

def find_closest_card(training, img):
    features = preprocess(img)
    return sorted(
        training.values(),
        key=lambda x: imgdiff(
            x[1],
            features))[0][0]

###############################################################################
# Card Extraction
###############################################################################
def getCards(im, numcards=4):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (1, 1), 1000)
    flag, thresh = cv2.threshold(blur, 120, 255, cv2.THRESH_BINARY)

#  contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    _, contours, hierarchy = cv2.findContours(
        thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:numcards]

    for card in contours:
        peri = cv2.arcLength(card, True)
        approx = rectify(cv2.approxPolyDP(card, 0.02 * peri, True))
        
#        if __name__ == '__main__':
#            box = np.int0(approx)
#            cv2.drawContours(im,[box],0,(255,255,0),6)
#            imx = cv2.resize(im,(1000,600))
#            cv2.imshow('a',imx)
#            cv2.waitKey(0)
#           cv2.destroyAllWindows()


        h = np.array([[0, 0], [449, 0], [449, 449], [0, 449]], np.float32)
        #Affine transformation
        transform = cv2.getPerspectiveTransform(approx, h)
        warp = cv2.warpPerspective(im, transform, (450, 450))
        yield warp
        
def get_training(
        training_labels_filename,
        training_image_filename,
        num_training_cards,
        avoid_cards=None):

    training = {}
    labels = {}

#  for line in file(training_labels_filename):
#    key, num, suit = line.strip().split()
#    labels[int(key)] = (num,suit)
    print("Reading training images")
    start_time = time.clock()
    with open(training_labels_filename, newline='') as fp:
        reader=csv.DictReader(fp,delimiter='\t')
        for i,row in enumerate(reader):
             key_ = str(row['Key'])
             num_ = str(row['Num'])
             suit_= str(row['Suit'])
             labels[int(key_)] = (num_, suit_)
    print("Done reading")
    print("--- %s seconds ---" % (time.clock() - start_time))
    fp.close()
    print("Training")
    start_time = time.clock()
    im = cv2.imread(training_image_filename)
    for i, c in enumerate(getCards(im, num_training_cards)):
#        cv2.imwrite(labels[i][0]+labels[i][1]+".jpg",c)
        if avoid_cards is None or (
                labels[i][0] not in avoid_cards[0] and labels[i][1] not in avoid_cards[1]):
            training[i] = (labels[i], preprocess(c))

    print("Done training!!!!!!!!!!")
    print("--- %s seconds ---" % (time.clock() - start_time))
    return training

In [2]:
training = get_training("train.tsv","train.png",56)

Reading training images
Done reading
--- 0.003139000000000003 seconds ---
Training
Done training!!!!!!!!!!
--- 0.8357059999999998 seconds ---


In [3]:
def run24(filename,num_cards, training):
    im=cv2.imread(filename)
    
    width = im.shape[0]
    height = im.shape[1]
    if width < height:
        im = cv2.transpose(im)
        im = cv2.flip(im, 1)
    
    cards=[]
    # Debug: uncomment to see registered images
    for i,c in enumerate(getCards(im,num_cards)):
        card = find_closest_card(training,c,)
#        cv2.imshow("?",c)
        cv2.imshow(str(card[0])+str(card[1]),c)
        cv2.waitKey(1500)
        cards.append(card)

#    cards = [find_closest_card(training,c) for c in getCards(im,num_cards)]
    print(cards)
    
    xlist=[]
    for x,_ in cards:
        x=str(x)
        if (x =="A"):
            xlist.append("1")
        elif (x in "12345678910"):
            xlist.append(x)
        else:
            print("Invaild card!")
            quit()
    compute(xlist)
    cv2.destroyAllWindows()

In [15]:
run24("test.jpg",4, training)

[('8', 'S'), ('7', 'H'), ('3', 'H'), ('6', 'S')]

Can you find 24 ? Hit Enter key to see the answer.
Number of answers is : 122
['8', '7', '+', '3', '+', '6', '+']
['8', '7', '+', '3', '6', '+', '+']
['8', '7', '3', '+', '6', '+', '+']
['8', '7', '3', '6', '+', '+', '+']
['8', '7', '-', '3', '+', '6', '*']
['8', '7', '+', '6', '+', '3', '+']
['8', '7', '+', '6', '3', '+', '+']
['8', '7', '6', '+', '3', '+', '+']
['8', '7', '6', '3', '+', '+', '+']
['8', '7', '6', '-', '3', '*', '*']
['8', '3', '+', '7', '+', '6', '+']
['8', '3', '+', '7', '6', '+', '+']
['8', '3', '7', '+', '6', '+', '+']
['8', '3', '7', '6', '+', '+', '+']
['8', '3', '+', '7', '-', '6', '*']
['8', '3', '7', '6', '-', '*', '*']
['8', '3', '7', '6', '-', '/', '*']
['8', '3', '*', '7', '6', '-', '*']
['8', '3', '*', '7', '6', '-', '/']
['8', '3', '+', '6', '+', '7', '+']
['8', '3', '+', '6', '7', '+', '+']
['8', '3', '6', '+', '7', '+', '+']
['8', '3', '6', '7', '+', '+', '+']
['8', '6', '+', '7', '+', '3', '+']
['8', '6

In [3]:
img = cv2.imread("9S.jpg",0)
#ret, thresh = cv2.threshold(img, 10, 255, cv2.THRESH_OTSU)

#print ("Threshold selected : ", ret)

(h, w) = img.shape[:2]
center = (w / 2, h / 2)
 
# rotate the image by 180 degrees
M = cv2.getRotationMatrix2D(center, 180, 1.0)
rotated = cv2.warpAffine(img, M, (w, h))
cv2.imshow("rotated", rotated)
cv2.waitKey(0)
cv2.imshow("original",img)
cv2.waitKey(0)

#cv2.imshow(' ',thresh)
#cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
import pytesseract
import os
from PIL import Image, ImageEnhance, ImageFilter
im = Image.open("9S.jpg")
#im = Image.open("Screen1.png") # the second one 
im = im.filter(ImageFilter.MedianFilter())
enhancer = ImageEnhance.Contrast(im)
im = enhancer.enhance(2)
im = im.convert('1')

cimg = im.crop((0, 0, 100, 70))
cimg.show()
#cv2.imshow("corner",cimg)
#img = cv2.imread("9S.jpg")
#gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#blur = cv2.GaussianBlur(gray, (1, 1), 1000)
#flag, thresh = cv2.threshold(blur, 120, 255, cv2.THRESH_BINARY)
#_, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#cnt = contours[0]
#r = cv2.boundingRect(cnt)
#cv2.imshow('corner', img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]])

#cv2.imshow("cropped", crop_img)
#cv2.waitKey(0)

#cimg.save('temp2.jpg')
#timg=Image.open('temp2.jpg')
text = pytesseract.image_to_string(im)

print(text)

#cv2.destroyAllWindows()

26%
have
